In [ ]:
file = 'data/test/pmc_simvastatin_amiodarone_filtered_128.pubtator'

i = 0
with open(file, 'r') as f:
    line_buffer = []
    for l in f:
        line_buffer.append(l)
        # split file here
        if l == '\n':
            fo_name = '{}.{}'.format(file,i)
            with open(fo_name, 'w') as fo:
                for lo in line_buffer:
                    fo.write(lo)
            print('{} written'.format(fo_name))
            line_buffer = []
            i += 1

In [ ]:
from snorkel import SnorkelSession
from snorkel.parser import CorpusParser, Spacy, StanfordCoreNLPServer
from pubtator import PubTatorDocPreprocessor, PubTatorTagProcessor, PubTatorParser
from time import time

%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

parser = "spacy"
num_procs = 1



for i in range(74, 75):
    print('<<<<<< Process Document {}  >>>>>>>'.format(i))
    start_ts = time()
    file_name = '{}.{}'.format(file, i)
    doc_preprocessor = PubTatorDocPreprocessor(file_name, annotations=False, debug=True)
    #arser = Spacy() if parser == "spacy" else StanfordCoreNLPServer()
    parser = PubTatorParser(stop_on_err=False)
    corpus_parser = CorpusParser(parser=parser)
    corpus_parser.apply(doc_preprocessor, parallelism=num_procs, clear=True)
    end_ts = time()
    
    from snorkel.models import Document, Sentence

    print("Documents:", session.query(Document).count())
    print("Sentences:", session.query(Sentence).count())

    print("Loading all sentences from db...")
    all_sents = session.query(Sentence).all()
    print("Loading complete!")


    print('Amount of sentences: {}'.format(len(all_sents)))

    print("\nDONE in {}".format((time() - start_ts)))
    
    from snorkel.models import Candidate, candidate_subclass
    from snorkel.candidates import PretaggedCandidateExtractor

    ChemicalGeneInteraction = candidate_subclass('ChemicalGeneInteraction', ['chemical', 'gene'])
    candidate_extractor = PretaggedCandidateExtractor(ChemicalGeneInteraction, ['Chemical', 'Gene'])

    for k, sents in enumerate([all_sents]):
        candidate_extractor.apply(sents, split=k, clear=True)
        print("Number of candidates:", session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == k).count())

    